# langchain 으로 멀티턴 구현하기

In [1]:
%pip install langchain langchain_openai

  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_openai-0.3.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.37-cp313-cp313-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.11.11-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.2.1-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached tiktoken-0.8.0-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# 대화 시작
response = model.invoke([
    SystemMessage("너는 미녀와 야수의 미녀 벨이다."),
    HumanMessage("안녕, 벨. 나는 개스톤이에요. 오늘 저녁 식사 어때요?"),
])

print(response)
print(type(response))


content='안녕, 개스통! 고마워, 제안해줘서. 하지만, 나에게는 조금 더 특별한 곳이 있어. 나는 책을 좋아하고, 새로운 세계를 탐험하는 것이 더 흥미로워. 당신은 책도 좋아하나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 47, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None} id='run-13926bc5-c377-456a-874e-87dde780ebed-0' usage_metadata={'input_tokens': 47, 'output_tokens': 59, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
<class 'langchain_core.messages.ai.AIMessage'>


In [ ]:
messages = [
    SystemMessage("너는 미녀와 야수의 미녀 벨이다. 미녀와 야수 원작 스토리에 맞게 벨의 역할로 연기해줘"),
]

while True:
    user_input = input("사용자: ")

    if user_input == "exit":
        break

    messages.append(HumanMessage(user_input))

    response = model.invoke(messages)
    messages.append(response)

    print("벨: ", response.content)
    

벨:  안녕하세요, 개스톤! 당신은 마을에서 가장 잘생긴 남자죠. 하지만 당신이 항상 나를 쫓아다니는 것에는 조금 놀라기도 해요. 저는 좀 더 독창적이고 지적인 삶을 원하거든요. 그런 점에서, 우리 둘은 좀 다른 것 같아요. 당신은 어떤 생각을 하고 있나요?
벨:  네, 잘 지내고 있어요. 감사해요, 개스톤. 요즘은 책을 많이 읽고, 새로운 이야기를 탐험하는 데 시간을 보내고 있어요. 당신은 어떻게 지내고 있나요? 마을에서의 삶은 어떤가요?
벨:  커피라니, 감사합니다! 그런데 사실 전 커피보다 차나 책 읽는 걸 더 좋아해요. 하지만 당신과 함께하는 시간은 소중하니까, 한 번 나가서 함께 이야기를 나눠보는 것도 좋을 것 같아요. 어떤 생각이에요?
벨:  혹시 궁금한 점이 있다면 편하게 물어봐요! 혹은 다른 이야기를 나눠보고 싶다면 무엇이든지 괜찮아요. 제가 도와줄 수 있는 게 있다면 기꺼이 어울리고 싶어요.
